Goal: Explore Data Provided by WDL on Urban Mobility

In [ ]:
# import necessary packages
import os
import pandas as pd
import numpy as np
from scipy.special import boxcox1p
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
# build data directory
data_dir = '../../data/stage1'
os.listdir(data_dir)

### 1: Churn User Profile Data
We will be examining this first

In [ ]:
# Load user profiles
user_profiles_df = pd.read_csv(
    os.path.join(data_dir, 'Churn_UsersProfile.txt'), 
    delimiter = "|",
    encoding = "ISO-8859-1"
)

#### 1.1: Inspect Data

In [ ]:
# sample some user profiles
user_profiles_df.sample(5)

In [ ]:
# No missing data -> Fantastic
user_profiles_df.isna().sum()

In [ ]:
user_profiles_df.info()

In [ ]:
# no duplicate rows!
user_profiles_df.duplicated().sum()

#### 1.2: Basic Data Wrangling
-  Make a new column for R1/R2 from Region_of_Origin
-  Make a new column for Region of origin anything after `- AM`
-  Convert columns to categorical
-  Separate period to `period_start`, `period_end` and `period_length`

Assumptions:
-  Period start is on the first day of the month and up to the final day in the last month

In [ ]:
# make new columns from Region_of_Origin
user_profiles_df['origin_region_number'] = user_profiles_df['Region_of_Origin'].str.split(' ').str[0]
user_profiles_df['origin_region'] = user_profiles_df['Region_of_Origin'].str.split('- AM',1).str[1]
user_profiles_df.drop('Region_of_Origin', axis=1, inplace=True)

In [ ]:
# # split period to period_start and period_end (assume from first day of month to last)
# user_profiles_df['period_start'] = user_profiles_df['Period'].str.split('to').str[0]
# user_profiles_df['period_end'] = user_profiles_df['Period'].str.split('to',1).str[-1]
# user_profiles_df.drop('Period', axis=1, inplace=True)

In [ ]:
# from datetime import datetime, timedelta

# user_profiles_df['period_start'] = user_profiles_df['period_start'].apply(lambda x: datetime.strptime(x.strip(), '%b-%y'))
# user_profiles_df['period_end'] = user_profiles_df['period_end'].apply(lambda x: datetime.strptime(x.strip(), '%b-%y'))

# def last_day_of_month(date):
#     if date.month == 12:
#         return date.replace(day=31)
#     return date.replace(month=date.month+1, day=1) - timedelta(days=1)

# user_profiles_df['period_end'] = user_profiles_df['period_end'].apply(last_day_of_month)

In [ ]:
# There are only two periods that we have; seems like wrangling dates wasn't necessary
user_profiles_df['Period'].value_counts()

In [ ]:
# convert types to categorical
to_cat_cols = ['District_of_Origin','County_of_Origin','Period','GenderDescription','AgeClassDescription','origin_region_number','origin_region']
for col in to_cat_cols:
    user_profiles_df[col] = user_profiles_df[col].astype('category')

#### 1.3: Plots

In [ ]:
# Lineplots of Average Bus Users Per Day
fig, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(8,5))
fig.suptitle('Lineplots of Bus Users Per Day')

ax1.plot(user_profiles_df['Average_BusUsers_per_Day']);
ax1.set(title='Average Bus Users Per Day');

ax2.plot(boxcox1p(user_profiles_df['Average_BusUsers_per_Day'], 0.15));
ax2.set(title='Boxcox Transforation of Bus Users Per Day');
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 5), constrained_layout=True)
fig.suptitle('Histplots of Bus Users Per Day')

sns.histplot(user_profiles_df['Average_BusUsers_per_Day'], ax=axs[0]);
axs[0].set(title='Histplot of Average Bus Users Per Day');

sns.histplot(boxcox1p(user_profiles_df['Average_BusUsers_per_Day'], 0.15), ax=axs[1]);
axs[1].set(title='Histplot of Average Bus Users Per Day Boxcox Tfms');

sns.histplot(np.log1p(user_profiles_df['Average_BusUsers_per_Day']), ax=axs[2]);
axs[2].set(title='Histplot of Average Bus Users Per Day log1p Tfms');
plt.show()

Looks like applying a log1p transformation transforms the distribution to a normal one; we'll do that and plot with it.

In [ ]:
user_profiles_df['log1p_avg_daily_bus_users'] = np.log1p(user_profiles_df['Average_BusUsers_per_Day'])

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='GenderDescription', y='log1p_avg_daily_bus_users',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='AgeClassDescription', y='log1p_avg_daily_bus_users',data=user_profiles_df);

In [ ]:
# look at average bus users per day by gender
plt.figure(figsize=(9,6))
sns.boxplot(x='AgeClassDescription', y='log1p_avg_daily_bus_users', hue='GenderDescription',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='Period', y='log1p_avg_daily_bus_users', hue='AgeClassDescription',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='origin_region', y='log1p_avg_daily_bus_users', hue='AgeClassDescription',data=user_profiles_df);

In [ ]:
sns.relplot(
    y='County_of_Origin', 
    x='log1p_avg_daily_bus_users', 
    hue='District_of_Origin', 
    size='log1p_avg_daily_bus_users',
    style='origin_region',
    sizes=(15,200),
    data=user_profiles_df,
    height=8
);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='District_of_Origin', y='log1p_avg_daily_bus_users', hue='AgeClassDescription',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='District_of_Origin', y='log1p_avg_daily_bus_users',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='District_of_Origin', y='log1p_avg_daily_bus_users', hue='GenderDescription',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='AgeClassDescription', y='log1p_avg_daily_bus_users', hue='Period',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(8,15))
sns.boxplot(x='log1p_avg_daily_bus_users', y='County_of_Origin', hue='District_of_Origin',data=user_profiles_df, orient='h');

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='GenderDescription', y='log1p_avg_daily_bus_users', hue='Period',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(9,6))
sns.boxplot(x='District_of_Origin', y='log1p_avg_daily_bus_users', hue='Period',data=user_profiles_df);

In [ ]:
plt.figure(figsize=(15,20))
sns.boxplot(y='County_of_Origin', x='log1p_avg_daily_bus_users', hue='Period',data=user_profiles_df, orient='h');

Findings:
-  Males and females have similar median, upper and lower ranges however there are more extreme cases for female bus users
-  Elderly (65+) have the highest median for usage of busses; middle age (35-44) have the lowest median
-  Female middle age have the lowest bus usage in age range (35-44) and slightly higher than males in age range (15-24)
-  There were slightly more users from Sep-2020 to Jan 2021 than Sep-19 to Feb-20
    -  Slight confusion as Pandemic hit Europe around March 2020? So Sep-20 to Jan-2021 is during the pandemic. Do we discount this information? <br>
    Response: Recover some "normality"; Sep-19 to Feb-20 is pre-pandemic and after is post pandemic however no longer under national lockdown so there is still some transportation
    -  Age demographics remain consistent for both of the time periods; elderly rely heavily on public transport
-  Gondomar and Maia have the highest avg_daily_bus_users
-  District of Origin makes a big difference; ages within that vary widely too; Aveiro have the lowest number of daily bus users and Famels from Aveiro even lower
-  Average daily bus users is down from first period to second period. Perhaps this is due to COVID-19, which is the main suspicion at the moment. Though in theory it should be much lower if it were actually COVID? Issue with the data?
-  More people from R2
-  Both male and females used less public transport after each period
-  Usage of public transportation decreased across districts and county of origins; perhaps we can look at these subsets to investigate what has happened

#### 1.4: More Investigating subgroups within each Period (before and After)

In [ ]:
sns.catplot(
    y='AgeClassDescription', 
    x='log1p_avg_daily_bus_users', 
    hue='Period',
    kind='violin',
    inner='stick',
    split=True,
    palette='pastel',
    data=user_profiles_df
);

In [ ]:
def show_values_on_bars(axs, h_v="v", space=0.4):
    '''
    https://stackoverflow.com/questions/43214978/seaborn-barplot-displaying-values
    '''
    def _modify_nan(value, replacement=0.1):
        if np.isnan(value):
            return replacement
        return value
    
    def _show_on_single_plot(ax):
        if h_v == "v":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() / 2
                _y = p.get_y() + p.get_height()
                value = int(p.get_height())
                value = _modify_nan(value)
                ax.text(_x, _y, value, ha="center") 
        elif h_v == "h":
            for p in ax.patches:
                _x = p.get_x() + p.get_width() + float(space)
                _y = p.get_y() + p.get_height()
                value = round((p.get_width()),2)
                value = _modify_nan(value)
                ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)

def facegrid_show_value_bars(g, h_v='v'):
    '''
    https://stackoverflow.com/questions/41127841/how-to-annotate-bars-in-a-seaborn-facetgrid-works-in-factorplot
    '''
    for ax in g.axes.ravel():
        show_values_on_bars(ax, h_v)
    plt.show()

In [ ]:
graph = sns.catplot(
    y='AgeClassDescription', 
    x='log1p_avg_daily_bus_users', 
    col='Period',
    kind='bar',
    palette='pastel',
    data=user_profiles_df
);
facegrid_show_value_bars(graph, h_v='h')

In [ ]:
graph = sns.catplot(
    y='GenderDescription', 
    x='log1p_avg_daily_bus_users', 
    col='Period',
    hue='AgeClassDescription',
    kind='bar',
    palette='pastel',
    data=user_profiles_df
);
facegrid_show_value_bars(graph, h_v='h')

In [ ]:
graph = sns.catplot(
    y='District_of_Origin', 
    x='log1p_avg_daily_bus_users', 
    col='Period',
    kind='bar',
    palette='pastel',
    data=user_profiles_df
);
facegrid_show_value_bars(graph, h_v='h')

In [ ]:
graph = sns.catplot(
    y='District_of_Origin', 
    x='log1p_avg_daily_bus_users', 
    col='Period',
    hue='AgeClassDescription',
    kind='bar',
    palette='pastel',
    data=user_profiles_df
);
facegrid_show_value_bars(graph, h_v='h')

In [ ]:
graph = sns.catplot(
    y='County_of_Origin', 
    x='log1p_avg_daily_bus_users', 
    col='Period',
    kind='bar',
    palette='pastel',
    data=user_profiles_df,
    height=13
);
facegrid_show_value_bars(graph, h_v='h')

Findings:
-  Across the board numbers decreased
-  55 and Upwards from Aveiro began using public transport in the second period
-  People from Arouca, Sao Joao da Madeira and Vale De Cambra began using public transport where there was no usage before
-  Very few places increased numbers from before and after; these are the signals we should look for retention
-  For all the places that have reported a decrease; we should look to find common trends among them

In [ ]:
# user_profiles_df

### 2: Churn Origin-Destination (OD) Data
Notes:
-  There are no time periods for the churn OD data; would we assume that the routes have not changed across the two different time periods?
-  Vinay has done some analysis; primary findings:
    -  is that Lisboa is the primary hub with several routes leading into it
    -  there are many routes that have the same origin and destination
    -  Q: Routes that start in one city and end in another but traverse through intermediary cities; these connect via other routes?

In [ ]:
# Load churn data
churn_od_df = pd.read_csv(
    os.path.join(data_dir, 'Churn_OD.txt'), 
    delimiter = "|",
    encoding = "ISO-8859-1"
)

#### 2.1: Inspect data
Notes:
-  Origin has region, district, county
-  Destination ends in 'of_Public_Transportation'
-  'Dicofre_ParishCode_of_Public_Transportation', short for 'DIstrito, COncelho, FREguesia' is the neighbourhood region
-  Demand_weight should sum to 1 for each county_origin

In [ ]:
churn_od_df.sample(5)

In [ ]:
# again, no missing data -> fantastic
churn_od_df.isna().sum()

In [ ]:
# 2253 rows
churn_od_df.info()

In [ ]:
churn_od_df.dtypes

In [ ]:
# no duplicate rows!
churn_od_df.duplicated().sum()

#### 2.2: Data wrangling
Data plots for discovery
-  Change naming '_Public_Transportation' to 'Destination'

In [ ]:
# change column titles
churn_od_df.columns = churn_od_df.columns.str.replace('_Public_Transportation', '_Destination')

In [ ]:
# plot barplots of each origin/destination
for col in churn_od_df.select_dtypes(include='object').columns:
    vcs = churn_od_df[col].value_counts()
    g = sns.barplot(data=churn_od_df, x=vcs.index.values, y=vcs.values)
    plt.title(col)
    show_values_on_bars(g)
    if 'County' in col:
        plt.xticks(rotation=90)
    plt.show()

In [ ]:
def move_legend(ax, new_loc, bbox_to_anchor=None, **kws):
    '''
    https://github.com/mwaskom/seaborn/issues/2280
    '''
    old_legend = ax.legend_
    handles = old_legend.legendHandles
    labels = [t.get_text() for t in old_legend.get_texts()]
    title = old_legend.get_title().get_text()
    if bbox_to_anchor is not None:
        ax.legend(handles, labels, loc=new_loc, bbox_to_anchor=bbox_to_anchor, title=title, **kws)
    else:
        ax.legend(handles, labels, loc=new_loc, title=title, **kws)

In [ ]:
# plot county with hues of district/region for origin/destination
fig, ax = plt.subplots(figsize=(12,9))
g = sns.histplot(data=churn_od_df, x='County_of_Origin', hue='District_of_Origin', ax=ax)
show_values_on_bars(g)
ax.tick_params(axis='x', labelrotation=90)
move_legend(ax, 'upper right', bbox_to_anchor=(1.15, 1))
ax.set_title('County Origins with District Hues');

In [ ]:
# plot county with hues of district/region for origin/destination
fig, ax = plt.subplots(figsize=(12,9))
g = sns.histplot(data=churn_od_df, x='County_of_Destination', hue='District_of_Destination', ax=ax)
show_values_on_bars(g)
ax.tick_params(axis='x', labelrotation=90)
move_legend(ax, 'upper right', bbox_to_anchor=(1.25, 1))
ax.set_title('County Destinations with District Hues');

In [ ]:
churn_od_df['Dicofre_ParishCode_of_Destination'].value_counts()

Findings:
-  Lisboa is a more popular region of origin
-  Porto and Lisboa are close in origin districts, Aveiro is the least popular
-  Counties vary quite widely in terms of origin locations; perhaps we can try to find out which counties belong to which districts to see if there's a common trend
-  Setubal is not even an origin but exists as a destination district 
    - TODO: Investigate what links exist there? Are there even transport links that go from there?
-  Most popular county is by far Lisboa, rest are dwarfed. Raises multiple questions about travel purposes and so on..
-  Aveiro has the least number of counties and one with the lowest number of origin passengers
-  Aveiro is not even a district for destination; and setubal has two destinations. Porto has quite a few by count but lisboa by far dominates
-  There are 100 parish codes; we will not work with these for now but could be valuable to look at later on.

#### 2.3: Visualising types of routes (internal v external)

In [ ]:
# breakdown of internal v external routes
churn_od_df['County_of_Origin'] = churn_od_df['County_of_Origin'].str.upper()
routes_count = (churn_od_df.groupby('County_of_Origin')
                           .agg({"County_of_Origin": 'count'})
                           .rename(columns={'County_of_Origin': 'number_of_routes'})
                           .sort_values(by=['number_of_routes'], ascending = False))

churn_od_df_internal = churn_od_df[churn_od_df['County_of_Destination'] == churn_od_df['County_of_Origin']]
churn_od_df_internal_agg = (churn_od_df_internal.groupby('County_of_Origin')
                    .agg({'County_of_Origin': 'count'})
                    .rename(columns={'County_of_Origin': 'number_of_routes_internal'}))
            

routes_count_final = pd.merge(routes_count, churn_od_df_internal_agg, left_index=True, right_index=True,  how = 'inner')
routes_count_final['percentage_of_routes_internal'] = routes_count_final['number_of_routes_internal']/ routes_count_final['number_of_routes']
routes_count_final['number_of_routes_external'] =  routes_count_final['number_of_routes'] - routes_count_final['number_of_routes_internal']
routes_count_final['percentage_of_routes_external'] =  1-  routes_count_final['percentage_of_routes_internal']
routes_count_final

In [ ]:
# plot number of internal vs external routes
routes_count_final[
    ['number_of_routes_internal', 'number_of_routes_external']
    ].plot.bar(rot=30, color=['darkgreen', 'crimson'], figsize=(15,10))
plt.title('Number of Routes Internal vs External');

In [ ]:
# plot percentage vs external routes
routes_count_final[
    ['percentage_of_routes_internal', 'percentage_of_routes_external']
    ].plot.bar(
    rot=30, color=['darkgreen', 'crimson'], figsize=(15,10), stacked=True
)
plt.title('Percentage of Routes Internal vs External');

In [ ]:
# plot a heatmap
fig, ax = plt.subplots(figsize=(14,14))  
sns.heatmap(
    pd.crosstab(
        churn_od_df.County_of_Origin, 
        churn_od_df.County_of_Destination
    ),
    annot=True,
    cmap='RdYlGn_r', 
    linewidths=0.5,
    ax=ax
);

In [ ]:
# finally, parish codes
fig, ax = plt.subplots(figsize=(15,25))
sns.countplot(
    y='Dicofre_ParishCode_of_Destination', 
    hue='County_of_Destination',
    dodge=False,
    data=churn_od_df,
);
move_legend(ax, 'upper right', bbox_to_anchor=(1.25, 1))
ax.set_title('Parish Codes for Destinations colored by District Hues');

Findings:
-  There are far more routes going out than there are going in
-  Lisboa has the highest number of routes going in
    -  Does this mean all of the other places are going to Lisboa?
    -  Pretty much verified by the heatmap.. Perhaps we can build a graph network next but the heatmap depicts what is happening
-  Parish codes give us more granularity but we only have the destinations.. Can we get the shapefiles or latitude/longitude of these to map out and understand the network effect of what is happening?

#### 2.4 Sankey Diagrams
At the county level

In [ ]:
# for plotting sankey diagrams
import plotly.graph_objects as go
import plotly.express as pex
import holoviews as hv

In [ ]:
hv.extension('bokeh')

In [ ]:
origin_destination_county_df = (churn_od_df.groupby(['County_of_Origin','County_of_Destination'])
           .agg({'Demand_weight': 'sum'}))
origin_destination_county_df = origin_destination_county_df.reset_index()

In [ ]:
origin_destination_county_df['County_of_Destination'] = origin_destination_county_df['County_of_Destination'] + '_1'

In [ ]:
# everythingg
hv.Sankey(origin_destination_county_df).options(height=3000)

In [ ]:
# plot county by county
for unique_county in origin_destination_county_df['County_of_Origin'].unique():
    tmp_df = origin_destination_county_df[origin_destination_county_df['County_of_Origin']==unique_county]
    tmp_df = tmp_df[tmp_df['Demand_weight'] > 0.0001]
    display(hv.Sankey(tmp_df).options(width=800, height=500))

Findings
-  Most places go to Lisboa
-  Espinho, Gondomar, Maia, Santa Maria De Feira goes to Porto mostly
-  There is a strong link between MAIA and Matosinhos
-  Many popular routes go to themself, ex. Lisboa
-  Big parts of routes often go to themself, ex. Oeiras
-  Paredes, Santo Tirso, Valongo is quite evenly split
-  Povoa De Varzim as a lot of weighting to Vila Nova De Gaia
-  Trofa is split between Porto and Lisboa

#### 2.4: Building a Graph Network
Numerically assess the hubs/clusters etc..

In [ ]:
import networkx as nx

In [ ]:
churn_od_df